In [156]:
import math
from collections import defaultdict
from collections import Counter

# You may not add any other imports

# For each function, replace "pass" with your code

# --- TASK 1: READING DATA ---

# 1.1
def read_ratings_data(f):
    movieRating = {}
    for line in open(f):
        movie, rating, userID = line.split("|")
        if movie not in movieRating:
            movieRating[movie.strip()] = []
        ratingList = movieRating.get(movie)
        ratingList.append(float(rating))
    return movieRating

# 1.2
def read_movie_genre(f):
    movieGenre = {}
    for line in open(f):
        genre, movieID, movie = line.split("|")
        if movie not in movieGenre:
            movieGenre[movie.strip()] = genre.strip()
    return movieGenre

# --- TASK 2: PROCESSING DATA ---

# 2.1
def create_genre_dict(d):
    genreDict = {}
    for movieKey in d.keys(): 
        if d[movieKey] not in genreDict: 
            genreDict[d[movieKey]] = []
        genreMovieList = genreDict.get(d[movieKey])
        genreMovieList.append(movieKey)
    return genreDict

# 2.2
def calculate_average_rating(d):
    averageRating = {}
    for movie in d.keys():
        total = 0
        count = 0
        for rating in d[movie]:
            total += rating
            count += 1
        averageRating[movie] = total/count
    return averageRating

# --- TASK 3: RECOMMENDATION ---

# 3.1
def get_popular_movies(d, n=10):
    popMoviesLH = {}
    popMoviesLH = dict(sorted(d.items(),
                      key= lambda movie: movie[1]))
    popMoviesHL = dict(reversed(list(popMoviesLH.items())))
    return dict(list(popMoviesHL.items())[0: n])

# 3.2
def filter_movies(d, thres_rating=3):
    filteredMovies = {}
    for k in d:
        if d[k]>=thres_rating: 
            filteredMovies[k] = d[k]
    return filteredMovies

# 3.3
def get_popular_in_genre(genre, genre_to_movies, movie_to_average_rating, n=50):
    genreMovies = []
    for k in genre_to_movies:
        if k.lower()==genre.lower(): 
            genreMovies.append(genre_to_movies[k])
    
    genreMoviesWAverages = {} 
    for k in movie_to_average_rating:
        if k in genreMovies[0]: 
            genreMoviesWAverages[k] = movie_to_average_rating[k]
    
    topNMoviesOfGenreLH = {}
    topNMoviesOfGenreLH = dict(sorted(genreMoviesWAverages.items(),
                      key= lambda movie: movie[1]))
    topNMoviesOfGenreHL = dict(reversed(list(topNMoviesOfGenreLH.items())))
    return dict(list(topNMoviesOfGenreHL.items())[0: n])
    

# 3.4
def get_genre_rating(genre, genre_to_movies, movie_to_average_rating):
    genreMovies = []
    for k in genre_to_movies:
        if k.lower()==genre.lower(): 
            genreMovies.append(genre_to_movies[k])
    
    genreMoviesWAverages = {} 
    for k in movie_to_average_rating:
        if k in genreMovies[0]: 
            genreMoviesWAverages[k] = movie_to_average_rating[k]
    
    total = 0
    for k in genreMoviesWAverages:
        total += genreMoviesWAverages[k]
    
    return total/len(genreMoviesWAverages)

# 3.5
def genre_popularity(genre_to_movies, movie_to_average_rating, n=5):
    genresWPop = {}
    for k in genre_to_movies:
        if k not in genresWPop.keys():
            genreRating = get_genre_rating(k, genre_to_movies,movie_to_average_rating)
            genresWPop[k] = genreRating
            
    genrePopLH = {}
    genrePopLH = dict(sorted(genresWPop.items(),
                      key= lambda movie: movie[1]))
    genrePopHL = dict(reversed(list(genrePopLH.items())))
    return dict(list(genrePopHL.items())[0: n])

# --- TASK 4: USER FOCUSED ---

# 4.1
def read_user_ratings(f):
    userMovieRatings = {}
    for line in open(f):
        movie, rating, userID = line.split("|")
        if userID.strip() not in userMovieRatings:
            userMovieRatings[userID.strip()] = []
        ratingList = userMovieRatings.get(userID.strip())
        
        ratingList.append([movie.strip(),float(rating)])
    return userMovieRatings

# 4.2
def get_user_genre(user_id, user_to_movies, movie_to_genre):
    userGenreWMovieRating = {}
    userMovies = []
    for k in user_to_movies:
        if int(k)==user_id:
            userMovies = user_to_movies[k]
    
    for mR in userMovies:
        movieGenre = movie_to_genre[mR[0]]
        if movieGenre not in userGenreWMovieRating:
            userGenreWMovieRating[movieGenre] = []
        genreMoviesList = userGenreWMovieRating.get(movieGenre)
        genreMoviesList.append(mR)
    
    genreWRating = {}
    for k in userGenreWMovieRating:
        genreMoviesList = userGenreWMovieRating.get(k)
        total = 0
        count = 0
        for movie in genreMoviesList:
            total += movie[1]
            count += 1
        genreAverage = total / count
        genreWRating[k] = genreAverage
    
    genreAnswer = ''
    rating = 0
    for k in genreWRating:
        if genreWRating[k]>=rating:
            genreAnswer = k
            rating = genreWRating[k]
   
    return genreAnswer
    

# 4.3
def recommend_movies(user_id, user_to_movies, movie_to_genre, movie_to_average_rating):
    userGenre = get_user_genre(user_id,user_to_movies,movie_to_genre)
    popInGenreMovies = get_popular_in_genre(userGenre, create_genre_dict(movie_to_genre), movie_to_average_rating, n=50)
    
    #print(user_to_movies)
    moviesWatched = user_to_movies.get(str(user_id))
    moviesWatchedDic = {}
    for movie in moviesWatched: 
        moviesWatchedDic[movie[0]] = movie[1]
    
    topRec = {}
    count = 0
    for k in popInGenreMovies: 
        if k not in moviesWatchedDic.keys():
            topRec[k] = popInGenreMovies[k]
            count += 1
        if count==3:
            break 
    return topRec
# --- main function for your testing ---
def main():
    movieRatings = read_ratings_data("movieRatingSample.txt")
    movieGenres = read_movie_genre("genreMovieSample.txt")
    
    genreDictionary = create_genre_dict(movieGenres)
    aveRating = calculate_average_rating(movieRatings)
    topNMovies = get_popular_movies(aveRating,10)
    filterMovieBasedOnRating = filter_movies(aveRating, thres_rating=3)
    topNOfGenre = get_popular_in_genre('adventure', genreDictionary, aveRating, n=5)
    genrePop = get_genre_rating('adventure', genreDictionary, aveRating)
    topNBasedOnGenre = genre_popularity(genreDictionary, aveRating,5)
    moviesRatedByUser = read_user_ratings("movieRatingSample.txt")
    userTopGenre = get_user_genre(11,moviesRatedByUser,movieGenres)
    userRec = recommend_movies(8,moviesRatedByUser,movieGenres,aveRating)

    ans.append((getMonthNum(arrayOfWords[0]).join('/').join(arrayOfWords[1]),(int(arrayOfWords[2]))))
main()


{'Toy Story (1995)': 3.8333333333333335, 'Tom and Huck (1995)': 2.8333333333333335}


In [25]:

def read_ratings_data(f,low,upper):
    ans = []
    for line in open(f):
        arrayOfWords = line.replace(':',' ').split()
        if(low <= int(arrayOfWords[2]) <= upper): 
            monthNum = getMonthNum(arrayOfWords[0])
            firstTuple = monthNum + "/" + arrayOfWords[1]
            ans.append((firstTuple,arrayOfWords[2]))
    return ans
        

def getMonthNum(month):
    if(month.lower()=='jan'): return '01'
    if(month.lower()=='feb'): return '02'
    if(month.lower()=='mar'): return '03'
    if(month.lower()=='apr'): return '04'
    if(month.lower()=='may'): return '05'
    if(month.lower()=='jun'): return '06'
    if(month.lower()=='jul'): return '07'
    if(month.lower()=='aug'): return '08'
    if(month.lower()=='sep'): return '09'
    if(month.lower()=='oct'): return '10'
    if(month.lower()=='nov'): return '11'
    if(month.lower()=='dec'): return '12'
    
def main():
    print(read_ratings_data("prob1txt.txt",30,80))
    
main()


[('03/15', '46'), ('01/2', '32'), ('09/09', '79'), ('04/5', '57')]


In [42]:
import string 
from collections import Counter
def wordCount(f):
    ans = []
    for line in open(f):
        changedLineWOPun = line.translate(str.maketrans('', '', string.punctuation))
        arrayWWords = changedLineWOPun.split()
        word_counts = Counter(arrayWWords)
        for word in arrayWWords:
            if(len(word)>=4):
                ans.append((word,word_counts[word]))
    return ans
        

    
    
def main():
    print(wordCount("prob2.txt"))
    
main()

[('this', 1), ('absolutely', 1), ('amazing', 1), ('crazy', 1), ('movie', 1), ('what', 1), ('funniest', 1), ('movie', 1), ('this', 1), ('year', 1)]


In [46]:
fours = [x for x in range(101) if x % 4 == 0]
print(fours)

[0, 4, 8, 12, 16, 20, 24, 28, 32, 36, 40, 44, 48, 52, 56, 60, 64, 68, 72, 76, 80, 84, 88, 92, 96, 100]


In [49]:
city_temp = [('sdf',45),('abc',88)]
ans = [city for (city,temp) in city_temp if temp>70]
print(ans)

['abc']


In [54]:
ans = [(x,y) for x in range (1,11) for y in range (1,11) if (x+y)%3==0]
print(ans)

[(1, 2), (1, 5), (1, 8), (2, 1), (2, 4), (2, 7), (2, 10), (3, 3), (3, 6), (3, 9), (4, 2), (4, 5), (4, 8), (5, 1), (5, 4), (5, 7), (5, 10), (6, 3), (6, 6), (6, 9), (7, 2), (7, 5), (7, 8), (8, 1), (8, 4), (8, 7), (8, 10), (9, 3), (9, 6), (9, 9), (10, 2), (10, 5), (10, 8)]


In [60]:
first = ['freshman','sophomore','junior','senior']
second = ['res','com']
ans = [for f in first and  
           for s in second ((f[0]),(s[0]))]
print(ans)

SyntaxError: invalid syntax (513968647.py, line 3)

In [13]:
import re
variableName = 'radar'
#res = re.match('[\s]*([\s]*[\S][\s]*)([\s]*[\S][\s]*)[\s]*[\S][\s]*\2[\s]*\1[\s]*', variableName)  
res = re.match('\([a-z]\)\([a-z]\)[a-z]\2\1', variableName)    
print(res)

None


In [49]:
import numpy as np
nparr = np.array([54,36,63,87])
count = 0
average = 0
total = 0
ans = []

for x in nparr:
    count+=1
    total+=x

average = total/count

for x in nparr: 
    if(x>=average):
        ans.append(x)
            
print(ans)

[63, 87]


In [31]:
import re 
test_str='18.0   8.   307.0      130.0      3504.      12.0   70.  1.	"chevrolet chevelle malibu"'

car_reg = re.compile(r"""
                \s*                 
                (?P<name>".*"),    
                (d{5}),    
                .*(?P<mpg>\d{1,2}\.\d{2})          
            """, re.VERBOSE)
res = car_reg.match('"Girl,Interrupted",1994,$11.99')
print(res)


None


In [32]:
^\s*(\"[^\"]*\"),(19[0-9]{2}|20[01][0-9]|20[2][01]),(\$[0-9]{1,2}\.[0-9]{2})\s*$
     

SyntaxError: invalid syntax (1168803514.py, line 1)

In [50]:
import re
variableName = 'rexer'
RegEx = r"[\s]*([\s]*[\S][\s]*)([\s]*[\S][\s]*)[\s]*[\S][\s]*\2[\s]*\1[\s]*"

strings = ['a1b1a', ' a 1b 1a', '#a?a # ', 'deed ', 'a1 b1 a']

for string in strings:
    if(re.search(RegEx, string)):
        print(f'{string} is a palindrome')
    else:
        print(f'{string} is not a palindrome')	

a1b1a is a palindrome
 a 1b 1a is a palindrome
#a?a #  is a palindrome
deed  is not a palindrome
a1 b1 a is a palindrome


In [ ]:
'\s*(\"[^\"]*\"),(d{4}),(\$d{1,2}\.\d{2})'